In [1]:
Dataset = "FB15K" # WN18 and FB15K
Embedding_size = 250 # 25,50,100,200,250
Epoch_count = 10 # fix record for 50 , 75 (for fb15k try for 50 75 takes time)
Learning_rate = 0.01 # fix



In [2]:
from google.colab import drive
drive.mount('/content/drive')
try:
  import torch
except:
  !pip install torch
try:
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version.
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric
    import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data

Mounted at /content/drive
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 87.0 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 53.6 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 73.9 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.1/932.1 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.3 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import torch
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
Dataset_path = "/content/drive/MyDrive/DATASETS/"+ Dataset
File_path = Dataset_path + "/edges_as_text_all.tsv"
df = pd.read_csv(File_path, sep='\t',header=None  ,names=["Subject","Predicate","Object"])
List = df.values.tolist()
#File_path = Dataset_path + "/edges_as_text_valid.tsv"
#df = pd.read_csv(File_path, sep='\t',header=None  ,names=["Subject","Predicate","Object"])
#List = List + df.values.tolist()


In [ ]:
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
num_features = Embedding_size

In [ ]:
Word2Vec_embeddings = Word2Vec(List,size=num_features, window=2, min_count=1, workers=4,sg=1)


In [ ]:
# reading a map_entity_id_to_text.tsv file and getting vector representation for a entity

File_path = Dataset_path + "/map_entity_id_to_text.tsv"
map_entity_id_to_text_features = pd.read_csv(File_path, sep='\t',header=None,names=["id","entity"])
#print(map_entity_id_to_text_features)
map_entity_id_to_text_features.set_index('id', inplace=True)

entity_features = []
for i in map_entity_id_to_text_features['entity']:
  entity_features.append(Word2Vec_embeddings.wv[i])


map_entity_id_to_text_features['entity_features'] = entity_features

n_entities = len(map_entity_id_to_text_features)

map_entity_id_to_text_features.head()

In [ ]:
# reading a map_entity_id_to_text.tsv file and getting vector representation for a entity

File_path = Dataset_path + "/map_relation_id_to_text.tsv"
map_relation_id_to_text_features = pd.read_csv(File_path, sep='\t',header=None,names=["id","relation"])
map_relation_id_to_text_features.set_index('id', inplace=True)

relation_features = []
for i in map_relation_id_to_text_features['relation']:
    relation_features.append(Word2Vec_embeddings.wv[i])

map_relation_id_to_text_features['relation_features'] = relation_features


n_relations = len(map_relation_id_to_text_features)

map_relation_id_to_text_features

In [ ]:
File_path = Dataset_path + "/edges_as_id_train.tsv"

Adjacency_list = []

edges_as_id_train = pd.read_csv(File_path, sep='\t',header=None  ,names=["Subject","Predicate","Object"])

Adjacency_list = torch.tensor([edges_as_id_train['Subject'],edges_as_id_train['Object']])

Adjacency_list

In [ ]:
File_path = Dataset_path + "/edges_as_id_test.tsv"

edges_as_id_test = pd.read_csv(File_path, sep='\t',header=None  ,names=["Subject","Predicate","Object"])


In [ ]:
File_path = Dataset_path + "/edges_as_id_valid.tsv"

edges_as_id_valid = pd.read_csv(File_path, sep='\t',header=None  ,names=["Subject","Predicate","Object"])

In [ ]:
from torch_geometric.nn import GATConv
import torch

class GAT(torch.nn.Module):
    def __init__(self,num_features):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_features, num_features,edge_dim=num_features)





    def forward(self, x, edge_index):


        # Message Passing Layer (Transformation)
        x = self.conv1(x, edge_index)
        x = x.relu()

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

GraphAttentionModel = GAT(num_features).to(device)

X = torch.tensor(entity_features)
X = X.to(device)

Adjacency_list = Adjacency_list.to(device)

X_updated = GraphAttentionModel(X, Adjacency_list)

In [ ]:
updated_node_features = X_updated.cpu()
updated_node_features = updated_node_features.detach().numpy()

In [ ]:
#updated_node_features = entity_features

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()


x_train = []
y_train = []

for triplet in edges_as_id_train.values:
    h,r,t = triplet[0],triplet[1],triplet[2]
    x_train.append(np.concatenate((updated_node_features[h],updated_node_features[t]),axis=None))

X_train = sc.fit_transform(x_train)
y_train = edges_as_id_train['Predicate'].values
y_train = tf.keras.utils.to_categorical(y_train, num_classes=n_relations)


In [ ]:
x_valid = []
y_valid = []

for triplet in edges_as_id_valid.values:
    h,r,t = triplet[0],triplet[1],triplet[2]
    x_valid.append(np.concatenate((updated_node_features[h],updated_node_features[t]),axis=None))

X_valid = sc.fit_transform(x_valid)

y_valid = edges_as_id_valid['Predicate'].values
y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=n_relations)

In [ ]:
edges_as_id_valid.values

In [ ]:
saved_model_name = "Scoring_Model" + "_" + Dataset + "_ES_" + str(Embedding_size) + "_EPC_" + str(Epoch_count)

print(saved_model_name)

In [ ]:
def get_hidden_units(input_layer,output_layer,hidden_layers):
  hidden_units = []

  if input_layer > output_layer :
    step = (input_layer - output_layer)/(hidden_layers+1)
    current = input_layer - step
    for i in range(hidden_layers):
      hidden_units.append(int(current))
      current = current - step
    return hidden_units
  else :
    return get_hidden_units(output_layer,input_layer,hidden_layers)[::-1]


In [ ]:
#get_hidden_units(500,18,4)

In [ ]:
import keras

from keras.layers import Dense , Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU
from keras. models import Sequential
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt

Neural_network_model = Sequential()



input = 2*Embedding_size
output = n_relations

hidden_units = get_hidden_units(input,output,4)

print(hidden_units)

Neural_network_model.add(Dense(hidden_units[0], activation='relu', input_dim= input))                     #input layer

#Neural_network_model.add(Dropout(rate=0.5))

Neural_network_model.add(Dense(hidden_units[1], activation='relu', input_dim= hidden_units[0]))           #1

#Neural_network_model.add(Dropout(rate=0.5))

Neural_network_model.add(Dense(hidden_units[2], activation='relu', input_dim= hidden_units[1]))           #2

#Neural_network_model.add(Dropout(rate=0.5))

Neural_network_model.add(Dense(hidden_units[3], activation='relu', input_dim= hidden_units[2]))           #3

#Neural_network_model.add(Dropout(rate=0.5))

Neural_network_model.add(Dense(output, activation= 'softmax', input_dim= hidden_units[3]))                #output layer


sgd=SGD(learning_rate=Learning_rate, decay=1e-6, momentum=0.9, nesterov=True)



Neural_network_model.summary()

Neural_network_model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
"""
history = Neural_network_model.fit(X_train,y_train,validation_data=(X_valid,y_valid),epochs=Epoch_count,batch_size=256,shuffle=True)


print(history.history)

plt.plot(history.history['accuracy'],label="accuracy")

plt.plot(history.history['loss'],label="loss")

plt.xlabel('Loss')

plt.ylabel('Iterations')

leg = plt.legend(loc='best', ncol=2, mode="expand", shadow=True, fancybox=True)
leg.get_frame().set_alpha(0.5)

plt.show()

del x_train
del y_train"""


In [ ]:

x_test = []
y_test = []
for triplet in edges_as_id_test.values:
    h,r,t = triplet[0],triplet[1],triplet[2]
    x_test.append(np.concatenate((updated_node_features[h],updated_node_features[t]),axis=None))

X_test = sc.transform(x_test)
#print(X_test)

#y_test = edges_as_id_train['Predicate'].values
#y_test = tf.keras.utils.to_categorical(y_train, num_classes=n_relations)

In [ ]:
def evaluate_model(test):
  j = 1
  results_c = []
  for i in range(1,Epoch_count+1,steps):
    Neural_network_model.fit(X_train,y_train,validation_data=(X_valid,y_valid),epochs=steps,batch_size=256,shuffle=True,verbose=False)
    pred_y  = Neural_network_model.predict(X_test)
    print("Epoch Count = ", j * steps)
    j+=1
    results = evaluation_metrics(edges_as_id_test['Predicate'],pred_y)
    #results_c.append(results)
    #test.__call__(results[0],Neural_network_model)
  return results_c


In [ ]:
#import numpy as np
#import torch
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=5, verbose=True, delta=0, path='checkpoint.pt'):

        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_mrr = None
        self.early_stop = False
        self.val_mrr_max = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_mrr, model):

        if self.best_mrr is None:
            self.best_mrr = val_mrr
            self.save_checkpoint(val_mrr, model)
        elif val_mrr < self.best_mrr + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_mrr = val_mrr
            self.save_checkpoint(val_mrr, model)
            self.counter = 0

    def save_checkpoint(self, val_mrr, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_mrr_max:.6f} --> {val_mrr:.6f}).  Saving model ...')
        #torch.save(model.state_dict(), self.path)
        model.save(self.path)
        self.val_mrr_max = val_mrr

In [ ]:
def Hit_n(ranks,n):
    count = 0
    for i in ranks:
        if i <= n:
            count = count + 1

    return count/len(ranks)*100

def Mean_rank(ranks):
    return sum(ranks)/len(ranks)

def Mean_reciprocal_rank(ranks):
    total_sum = 0
    for i in ranks:
        #print(1/i)
        total_sum = total_sum + (1/i)

    return total_sum/len(ranks)


def evaluation_metrics(actual_index_array,predicted_scores_array):
    rankings_array = []
    ranks = []
    count = 0
    for scores_array in predicted_scores_array:
        score_rankings = list(np.argsort(scores_array)[::-1])
        rankings_array.append(score_rankings)

        rank = score_rankings.index(actual_index_array[count]) + 1
        #print(scores_array,score_rankings,actual_index_array[count],rank)
        ranks.append(rank)
        count=count+1

    print("Hit@1 = "+"{:.2f}".format(Hit_n(ranks,1)))

    print("Hit@3 = "+"{:.2f}".format(Hit_n(ranks,3)))

    print("Hit@5 = "+"{:.2f}".format(Hit_n(ranks,5)))

    print("Hit@10 = "+"{:.2f}".format(Hit_n(ranks,10)))

    print("Mean Rank = "+"{:.2f}".format(Mean_rank(ranks)))

    print("Mean Reciprocal Rank = "+"{:.2f}".format(Mean_reciprocal_rank(ranks)))

    return [Mean_reciprocal_rank(ranks),Mean_rank(ranks),Hit_n(ranks,10),Hit_n(ranks,5),Hit_n(ranks,3),Hit_n(ranks,1)]

In [ ]:
#evaluation_metrics(edges_as_id_test['Predicate'],pred_y)

In [ ]:
"""#def evaluate_model():
pred_y  = Neural_network_model.predict(X_test)
evaluation_metrics(edges_as_id_test,pred_y)

"""

In [ ]:
#evaluate_model()

In [ ]:
test = EarlyStopping()
steps=1
results_c = evaluate_model(test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Epoch Count =  1
Hit@1 = 64.44
Hit@3 = 89.21
Hit@5 = 94.77
Hit@10 = 97.71
Mean Rank = 4.34
Mean Reciprocal Rank = 0.78
Epoch Count =  2
Hit@1 = 67.71
Hit@3 = 91.43
Hit@5 = 96.41
Hit@10 = 98.61
Mean Rank = 3.01
Mean Reciprocal Rank = 0.80
Epoch Count =  3
Hit@1 = 69.12
Hit@3 = 92.39
Hit@5 = 97.01
Hit@10 = 98.94
Mean Rank = 2.42
Mean Reciprocal Rank = 0.81
Epoch Count =  4
Hit@1 = 69.03
Hit@3 = 92.82
Hit@5 = 97.35
Hit@10 = 99.09
Mean Rank = 2.29
Mean Reciprocal Rank = 0.81
Epoch Count =  5
Hit@1 = 70.02
Hit@3 = 93.18
Hit@5 = 97.58
Hit@10 = 99.20
Mean Rank = 2.12
Mean Reciprocal Rank = 0.82
Epoch Count =  6
Hit@1 = 70.91
Hit@3 = 93.34
Hit@5 = 97.68
Hit@10 = 99.24
Mean Rank = 2.05
Mean Reciprocal Rank = 0.82
Epoch Count =  7
Hit@1 = 70.37
Hit@3 = 93.38
Hit@5 = 97.74
Hit@10 = 99.32
Mean Rank = 2.00
Mean Reciprocal Rank = 0.82
Epoch Count =  8
Hit@1 = 70.92
Hit@3 = 93.53
Hit@5 = 97.83
Hit@10 = 99.35
Mean Rank = 1.98
Mean Reciprocal Rank = 0.83
Epoch Count =  9
Hit@1 = 70.90
Hit@3 = 93.71
Hit